### VacationPy

In [54]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json 

# Import API key
from api_keys import geoapify_key

In [12]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
City_Data_DF = pd.read_csv("output_data/cities.csv")

# Display sample data
City_Data_DF.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


### Step 2: Narrow down the citi_data_df DataFrame to fiind your ideal weather condition

In [14]:
# Narrow down cities that fit criteria and drop any results with null values
City_DF = City_Data_DF[(City_Data_DF['Max Temp'] > 24) & 
                            (City_Data_DF['Max Temp'] < 30) & 
                            (City_Data_DF['Wind Speed'] < 10) &
                            (City_Data_DF['Cloudiness'] < 10 )].copy()

# Drop any rows with null values
City_DF.dropna(how='all', inplace=True)

# Display sample data
City_DF

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
51,51,hilo,19.7297,-155.0900,26.27,83,0,2.57,US,1666108260
63,63,banda,25.4833,80.3333,24.62,52,0,2.68,IN,1666108268
64,64,atuona,-9.8000,-139.0333,24.69,69,1,7.05,PF,1666108268
73,73,nalut,30.3333,10.8500,25.41,27,0,5.11,LY,1666108275
146,146,puerto del rosario,28.5004,-13.8627,24.86,73,0,4.63,ES,1666108035
197,197,gat,31.6100,34.7642,24.38,100,0,3.69,IL,1666108356
258,258,comodoro rivadavia,-45.8667,-67.5000,25.92,24,0,7.20,AR,1666108389
275,275,cockburn town,21.4612,-71.1419,28.18,74,6,5.43,TC,1666108398
317,317,salalah,17.0151,54.0924,25.49,59,7,1.67,OM,1666108424
340,340,santa rosalia,27.3167,-112.2833,24.62,56,0,0.74,MX,1666108436


### Step 3: Create a new DataFrame called hotel_df

In [16]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
Hotels_DF = City_DF[['City','Country','Lat', 'Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
Hotels_DF['Hotel Name']= ''

# Display sample data
Hotels_DF

,City,Country,Lat,Lng,Humidity,Hotel Name
51,hilo,US,19.7297,-155.0900,83,
63,banda,IN,25.4833,80.3333,52,
64,atuona,PF,-9.8000,-139.0333,69,
73,nalut,LY,30.3333,10.8500,27,
146,puerto del rosario,ES,28.5004,-13.8627,73,
197,gat,IL,31.6100,34.7642,100,
258,comodoro rivadavia,AR,-45.8667,-67.5000,24,
275,cockburn town,TC,21.4612,-71.1419,74,
317,salalah,OM,17.0151,54.0924,59,
340,santa rosalia,MX,27.3167,-112.2833,56,


### Step 4: For each city, use the Felapify API to find the first hotel located within 10,000 metres of your coordinates. 

In [40]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in Hotels_DF.iterrows():
    # get latitude, longitude from the DataFrame
    Lat = row.iloc[2]
    Lng = row.iloc[3]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Lng},{Lat},{radius}"
    params["bias"] = f"proximity:{Lng},{Lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        Hotels_DF.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        Hotels_DF.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{Hotels_DF.loc[index, 'City']} - nearest hotel: {Hotels_DF.loc[index, 'Hotel Name']}")

# Display sample data
Hotels_DF

Starting hotel search
hilo - nearest hotel: Dolphin Bay Hotel
banda - nearest hotel: #acnindiafy21
atuona - nearest hotel: Pearl Resort
nalut - nearest hotel: No hotel found
puerto del rosario - nearest hotel: Hotel Tamasite
gat - nearest hotel: No hotel found
comodoro rivadavia - nearest hotel: No hotel found
cockburn town - nearest hotel: The Salt Raker Inn
salalah - nearest hotel: Muscat International Hotel
santa rosalia - nearest hotel: Sol y Mar
pemba - nearest hotel: Hotel Rafael
quelimane - nearest hotel: Mabassa
capoterra - nearest hotel: Rosa Hotel
stolac - nearest hotel: Bregava
lakki marwat - nearest hotel: No hotel found
tikrit - nearest hotel: فندق بلازا
lahij - nearest hotel: No hotel found
bandarbeyla - nearest hotel: No hotel found
pali - nearest hotel: Hotel Pawan International


,City,Country,Lat,Lng,Humidity,Hotel Name
51,hilo,US,19.7297,-155.0900,83,Dolphin Bay Hotel
63,banda,IN,25.4833,80.3333,52,#acnindiafy21
64,atuona,PF,-9.8000,-139.0333,69,Pearl Resort
73,nalut,LY,30.3333,10.8500,27,No hotel found
146,puerto del rosario,ES,28.5004,-13.8627,73,Hotel Tamasite
197,gat,IL,31.6100,34.7642,100,No hotel found
258,comodoro rivadavia,AR,-45.8667,-67.5000,24,No hotel found
275,cockburn town,TC,21.4612,-71.1419,74,The Salt Raker Inn
317,salalah,OM,17.0151,54.0924,59,Muscat International Hotel
340,santa rosalia,MX,27.3167,-112.2833,56,Sol y Mar


### Step 5: Add the Hotel Name and the courty as additional information in the hover message for each city in the map. 

In [50]:
!pip install cartopy geoviews pyproj


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 34.8 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.3/547.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.5 MB/s eta 0:00:00


In [51]:
#Configure the map plot
Hotel_Map = Hotels_DF.hvplot.points("Lng", "Lat", geo= True,
                                   hover_cols=["Hotel Name", "Country"],
                                   size= 200, color= "City",      
                                   tiles= 'OSM', title = "Hotels", )                      
Hotel_Map.opts(width = 900, height = 500)
                                 
                        
                          
# Display the map
Hotel_Map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)